In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/data/raw_stock_ind.csv")

In [3]:
import json
df["info_json"] = df["info_json"].str.replace("\'", "\"")
df["info_json"] = df["info_json"].str.replace("None", "\"None\"")

In [4]:
def json_parser(x):
    print(x["info_json"])  
    return json.loads(x["info_json"])["symbol"]

In [5]:
df = df.assign(company_symbol=lambda x: json_parser(x))  

0       {"address1": "L&T House", "address2": "Ballard...
1       {"address1": "L&T House", "address2": "Ballard...
2       {"address1": "L&T House", "address2": "Ballard...
3       {"address1": "L&T House", "address2": "Ballard...
4       {"address1": "L&T House", "address2": "Ballard...
                              ...                        
1107    {"address1": "Fairfield by Marriott", "address...
1108    {"address1": "Fairfield by Marriott", "address...
1109    {"address1": "Fairfield by Marriott", "address...
1110    {"address1": "Fairfield by Marriott", "address...
1111    {"address1": "Fairfield by Marriott", "address...
Name: info_json, Length: 1112, dtype: object


TypeError: the JSON object must be str, bytes or bytearray, not Series

In [8]:
df.loc[10].company_symbol

AttributeError: 'Series' object has no attribute 'company_symbol'

In [ ]:
student_details["symbol"]